# Parameter Constraints and Bounds in Optimization

Real-world optimization problems often require **constraints** on parameters:
- **Physical limits**: Concentrations must be positive
- **Engineering constraints**: Dimensions within manufacturing tolerances
- **Mathematical requirements**: Probabilities sum to 1
- **Domain knowledge**: Reaction rates within known ranges

This notebook explores RustLab-Optimize's powerful constraint handling:
1. **Box bounds** - Simple upper/lower limits
2. **Parameter fixing** - Holding values constant
3. **Transformation techniques** - Log-space optimization
4. **Complex constraints** - Combining multiple requirements
5. **Algorithm behavior** - How solvers handle constraints

**Note**: This notebook uses a simplified approach to avoid rust-analyzer warnings.

## Setup

In [14]:
// Load required crates
:dep rustlab-math = { path = "../../rustlab-math" }
:dep rustlab-optimize = { path = ".." }

use rustlab_optimize::prelude::*;
use rustlab_math::linspace;

let msg = "Setup complete! Ready to explore constraints.";
println!("{}", msg);

Setup complete! Ready to explore constraints.


## 1. Box Bounds - Keeping Parameters in Valid Ranges

**Box bounds** are the simplest constraints: each parameter has a lower and upper limit.

$$\text{minimize } f(x) \text{ subject to } l_i \leq x_i \leq u_i$$

### Example: Chemical Reaction Rate

Fitting the power law: $\text{rate} = k \cdot [A]^n$
- Physical constraint: $k > 0$ (rate constant must be positive)
- Chemical constraint: $0 < n < 3$ (reaction order typically between 0 and 3)

In [15]:
// Experimental data: concentration vs reaction rate
{
    fn create_reaction_data() -> (Vec<f64>, Vec<f64>) {
        let concentrations = vec![0.1, 0.5, 1.0, 2.0, 5.0];
        let rates = vec![0.05, 0.22, 0.40, 0.71, 1.41];
        (concentrations, rates)
    }

    fn create_rate_model(concentrations: Vec<f64>, rates: Vec<f64>) -> impl Fn(&[f64]) -> f64 {
        move |params: &[f64]| {
            let k_val = params[0];
            let n_val = params[1];
            concentrations.iter().zip(rates.iter())
                .map(|(&conc, &rate)| {
                    let predicted = k_val * (conc as f64).powf(n_val);
                    (rate - predicted).powi(2)
                })
                .sum::<f64>()
        }
    }

    let (concentrations, rates) = create_reaction_data();
    let rate_model = create_rate_model(concentrations.clone(), rates.clone());

    let header = "Reaction Rate Data:";
    let subheader = "[A] (M)\tRate (M/s)";
    println!("{}", header);
    println!("{}", subheader);

    for (conc, rate) in concentrations.iter().zip(rates.iter()) {
        let line = format!("{:.1}\t{:.2}", conc, rate);
        println!("{}", line);
    }

    let model_info = "\nModel: rate = k * [A]^n";
    println!("{}", model_info);
    let success_msg = "Rate model defined successfully!";
    println!("{}", success_msg);
}

Reaction Rate Data:
[A] (M)	Rate (M/s)
0.1	0.05
0.5	0.22
1.0	0.40
2.0	0.71
5.0	1.41

Model: rate = k * [A]^n
Rate model defined successfully!


()

In [16]:
{
    // Re-define helper functions for this cell
    fn create_reaction_data() -> (Vec<f64>, Vec<f64>) {
        let concentrations = vec![0.1, 0.5, 1.0, 2.0, 5.0];
        let rates = vec![0.05, 0.22, 0.40, 0.71, 1.41];
        (concentrations, rates)
    }

    fn create_rate_model(concentrations: Vec<f64>, rates: Vec<f64>) -> impl Fn(&[f64]) -> f64 {
        move |params: &[f64]| {
            let k_val = params[0];
            let n_val = params[1];
            concentrations.iter().zip(rates.iter())
                .map(|(&conc, &rate)| {
                    let predicted = k_val * (conc as f64).powf(n_val);
                    (rate - predicted).powi(2)
                })
                .sum::<f64>()
        }
    }

    // Compare unbounded vs bounded optimization
    let (concentrations, rates) = create_reaction_data();

    // Unbounded optimization
    let unbounded_model = create_rate_model(concentrations.clone(), rates.clone());
    let unbounded_result = minimize(unbounded_model)
        .from(&[1.0, 1.0])
        .solve()?;

    // Bounded optimization  
    let bounded_model = create_rate_model(concentrations, rates);
    let bounded_result = minimize(bounded_model)
        .from(&[1.0, 1.0])
        .bounds(&[0.01, 0.1], &[10.0, 3.0])  // k ∈ [0.01, 10], n ∈ [0.1, 3]
        .solve()?;

    let unbounded_k = unbounded_result.solution[0];
    let unbounded_n = unbounded_result.solution[1];
    let bounded_k = bounded_result.solution[0];
    let bounded_n = bounded_result.solution[1];

    println!("Unbounded: k = {:.3}, n = {:.3}", unbounded_k, unbounded_n);
    println!("Bounded:   k = {:.3}, n = {:.3}", bounded_k, bounded_n);

    if unbounded_k < 0.0 {
        println!("Warning: Unbounded optimization gave negative rate constant");
    }

    println!("Bounds ensure physically meaningful parameters");
}

Unbounded: k = 0.014, n = -1.064
Bounded:   k = 0.010, n = 0.100
Bounds ensure physically meaningful parameters


()

## 2. Parameter Fixing - Reducing Problem Dimensionality

Sometimes we know certain parameter values from:
- **Prior experiments**
- **Physical measurements** 
- **Calibration standards**

Fixing these parameters:
- ✅ Reduces optimization complexity
- ✅ Improves convergence speed
- ✅ Prevents overfitting

### Example: Calibration Curve with Known Offset

In [17]:
{
    // Re-define helper functions for this cell
    fn create_calibration_data() -> (Vec<f64>, Vec<f64>) {
        let x_calib = vec![-2.0, -1.0, 0.0, 1.0, 2.0];
        let y_calib = vec![18.0, 11.5, 10.0, 12.5, 22.0];
        (x_calib, y_calib)
    }

    fn create_quadratic_fixed(x_data: Vec<f64>, y_data: Vec<f64>) -> impl Fn(&[f64]) -> f64 {
        move |params: &[f64]| {
            let a_val = params[0];
            let b_val = params[1];
            let c_val = 10.0;  // FIXED parameter
            
            x_data.iter().zip(y_data.iter())
                .map(|(&x, &y)| {
                    let predicted = a_val * x * x + b_val * x + c_val;
                    (y - predicted).powi(2)
                })
                .sum::<f64>()
        }
    }

    fn create_quadratic_full(x_data: Vec<f64>, y_data: Vec<f64>) -> impl Fn(&[f64]) -> f64 {
        move |params: &[f64]| {
            let a_val = params[0];
            let b_val = params[1];
            let c_val = params[2];  // FREE parameter
            
            x_data.iter().zip(y_data.iter())
                .map(|(&x, &y)| {
                    let predicted = a_val * x * x + b_val * x + c_val;
                    (y - predicted).powi(2)
                })
                .sum::<f64>()
        }
    }

    let (x_calib, y_calib) = create_calibration_data();

    // Fixed parameter optimization (2 variables)
    let fixed_model = create_quadratic_fixed(x_calib.clone(), y_calib.clone());
    let fixed_result = minimize(fixed_model)
        .from(&[1.0, 0.0])
        .solve()?;

    // Full optimization (3 variables)  
    let full_model = create_quadratic_full(x_calib, y_calib);
    let full_result = minimize(full_model)
        .from(&[1.0, 0.0, 10.0])
        .solve()?;

    let fixed_a = fixed_result.solution[0];
    let fixed_b = fixed_result.solution[1];
    let full_a = full_result.solution[0];
    let full_b = full_result.solution[1];
    let full_c = full_result.solution[2];

    println!("Fixed c=10.0: y = {:.3}x² + {:.3}x + 10.0", fixed_a, fixed_b);
    println!("Free c:       y = {:.3}x² + {:.3}x + {:.3}", full_a, full_b, full_c);
    println!("Fixed optimization used {} iterations", fixed_result.iterations);
    println!("Full optimization used {} iterations", full_result.iterations);

    let c_error = (full_c - 10.0).abs();
    if c_error < 0.01 {
        println!("Optimizer correctly found c ≈ 10.0");
    } else {
        println!("Optimizer found c = {:.3}, different from known value", full_c);
    }
}

Fixed c=10.0: y = 2.471x² + 0.900x + 10.0
Free c:       y = 2.571x² + 0.900x + 9.657
Fixed optimization used 6 iterations
Full optimization used 9 iterations
Optimizer found c = 9.657, different from known value


()

## 3. Wide Parameter Ranges and Transformations

When parameters span **multiple orders of magnitude** (e.g., $10^{-6}$ to $10^{3}$), standard optimization can struggle.

**Solution**: RustLab-Optimize uses **logarithmic transformations** internally:

$$x_{\text{unbounded}} = \log\left(\frac{x - l}{u - x}\right)$$

This transformation:
- Maps bounded domain $[l, u]$ to unbounded $(-\infty, +\infty)$
- Improves numerical stability
- Handles wide parameter ranges effectively

In [18]:
{
    // Re-define helper functions for this cell
    fn create_exponential_data() -> (Vec<f64>, Vec<f64>) {
        let t_decay = linspace(0.0, 5.0, 10);
        let true_a = 50.0;
        let true_k = 1.0;
        
        let y_decay: Vec<f64> = t_decay.to_vec().iter()
            .map(|&t| true_a * (-true_k * t).exp() + 0.5 * (t * 2.0).sin())
            .collect();
        
        (t_decay.to_vec(), y_decay)
    }

    fn create_decay_model(t_data: Vec<f64>, y_data: Vec<f64>) -> impl Fn(&[f64]) -> f64 {
        move |params: &[f64]| {
            let amp = params[0];
            let decay_rate = params[1];
            
            t_data.iter().zip(y_data.iter())
                .map(|(&t, &y)| {
                    let predicted = amp * (-decay_rate * t).exp();
                    (y - predicted).powi(2)
                })
                .sum::<f64>()
        }
    }

    let (t_decay, y_decay) = create_exponential_data();
    let decay_model = create_decay_model(t_decay, y_decay);

    // Optimize with wide bounds (6 orders of magnitude)
    let result = minimize(decay_model)
        .from(&[10.0, 0.5])
        .bounds(&[0.001, 0.01], &[1000.0, 10.0])  // A ∈ [0.001, 1000], k ∈ [0.01, 10]
        .tolerance(1e-10)
        .solve()?;

    let fitted_a = result.solution[0];
    let fitted_k = result.solution[1];

    println!("Wide Range Optimization:");
    println!("True:   A = 50.0, k = 1.0");
    println!("Fitted: A = {:.2}, k = {:.3}", fitted_a, fitted_k);
    println!("Converged in {} iterations", result.iterations);
    println!("Successfully handled 6 orders of magnitude range!");
}

Wide Range Optimization:
True:   A = 50.0, k = 1.0
Fitted: A = 44.58, k = 0.925
Converged in 1000 iterations
Successfully handled 6 orders of magnitude range!


()

## Summary and Best Practices

### Key Takeaways

1. **Box bounds** keep parameters physically meaningful
2. **Parameter fixing** reduces complexity and improves convergence  
3. **Transformations** handle wide parameter ranges automatically
4. **Complex constraints** can be implemented via penalties or reparameterization
5. **Algorithm choice** affects constraint handling behavior

### Best Practices

1. **Always use physical bounds** when parameters have natural limits
2. **Fix known parameters** to reduce search space
3. **Use penalties** for complex constraints like ordering
4. **Start with reasonable initial guesses** within constraint bounds
5. **Check constraint satisfaction** in final results

### Next Steps

- **[04_algorithm_selection.ipynb](04_algorithm_selection.ipynb)** - Deep dive into optimization algorithms
- **[05_scientific_applications.ipynb](05_scientific_applications.ipynb)** - Real-world scientific problems

**Try it yourself**: Apply these constraint techniques to your own optimization problems!

```rust
// Template for constrained optimization
let result = minimize(your_model)
    .from(&initial_guess)
    .bounds(&lower_bounds, &upper_bounds)
    .solve()?;
```